In [1]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import skimage
from skimage.transform import rescale
import rasterio
from shapely.geometry import box
import numpy as np
import rasterio
from glob import glob
from rasterio.warp import calculate_default_transform, reproject, Resampling
import os
import urllib.request
import requests
# import config
import json
import geopandas as gpd
import affine
import fiona
import argparse
import rasterio
from rasterio.merge import merge
import rasterio.mask
from rasterio.enums import Resampling
import geojson
import zipfile
import geojson
import geopandas as gpd
import requests
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from pyroSAR import identify_many
from pyroSAR import finder
import xml.etree.ElementTree as ET
import tarfile
import subprocess
import time
import math
import datetime
from scipy.io import loadmat, savemat
from tqdm import tqdm
import pprint
import os
from skimage.transform import rescale, resize, downscale_local_mean
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from rasterio.mask import mask
import scipy
import datetime
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [5]:
## Sentinel download
def downloadSentinel(aoi=None, path2save=None):
    if path2save is None:
        path2save = '/Sentinel-2_seond/Data'
    if aoi is None:
        aoi = '/Sky_SAT/footprint/explorer-aoi_final.geojson'
    api = SentinelAPI('user name', 'password', 'https://scihub.copernicus.eu/dhus/')  # change password by our login password
    # apihub.copernicus.eu/apihub
    # https://scihub.copernicus.eu/dhus/
    footprint = geojson_to_wkt(read_geojson(aoi))
    products = api.query(footprint,
                         date=('20190101', date(2019, 12, 28)),
                         platformname='Sentinel-2',
                         cloudcoverpercentage=(0, 10),
                         producttype='S2MSI1C') # processinglevel='Level-2A'
    api.download_all(products, directory_path=path2save)  # use path like this D:/EO_Africa_DAT/Sentinel-2_seond/Data

In [2]:
downloadSentinel(aoi=outfiles, path2save=path_s)

In [10]:
path = '/footprint/explorer-aoi_final.geojson'
footprint = geojson_to_wkt(read_geojson(path))
products = api.query(footprint,
                     date=('20221001', date(2022, 12, 12)),
                     platformname='Sentinel-2')

In [11]:
products_df = api.to_dataframe(products)
gdf = api.to_geodataframe(products)
gdf.plot()
gdf[['cloudcoverpercentage', 'processinglevel', 'endposition']]

In [46]:
gdf.keys()

Index(['title', 'link', 'link_alternative', 'link_icon', 'summary', 'ondemand',
       'generationdate', 'beginposition', 'endposition', 'ingestiondate',
       'orbitnumber', 'relativeorbitnumber', 'illuminationazimuthangle',
       'illuminationzenithangle', 'vegetationpercentage',
       'notvegetatedpercentage', 'waterpercentage', 'unclassifiedpercentage',
       'mediumprobacloudspercentage', 'highprobacloudspercentage',
       'snowicepercentage', 'cloudcoverpercentage', 'level1cpdiidentifier',
       'format', 'processingbaseline', 'platformname', 'filename',
       'instrumentname', 'instrumentshortname', 'size', 's2datatakeid',
       'producttype', 'platformidentifier', 'orbitdirection',
       'platformserialidentifier', 'processinglevel', 'datastripidentifier',
       'granuleidentifier', 'identifier', 'uuid', 'geometry'],
      dtype='object')

In [1]:
# api.download_all(products, directory_path='F:/EO_Africa_DAT/Sentinel-2_seond/Data')

In [8]:
path = '/Sky_SAT/footprint/explorer-aoi_final.geojson'
footprint = geojson_to_wkt(read_geojson(path))
s1_products = api.query(footprint,
                     date=('20220101', date(2022, 6, 1)),
                     platformname='Sentinel-1',
                        orbitdirection='DESCENDING',
                        polarisationmode='VV VH', slicenumber=7, relativeorbitnumber=152)

In [9]:
s1_products_df = api.to_dataframe(s1_products)
s1_products_df

,title,link,link_alternative,link_icon,summary,ondemand,beginposition,endposition,ingestiondate,missiondatatakeid,...,platformname,platformidentifier,instrumentname,instrumentshortname,productclass,polarisationmode,acquisitiontype,gmlfootprint,footprint,uuid
39f90581-99c2-4e50-843c-1d2c9ed2e5f8,S1A_IW_SLC__1SDV_20220531T031715_20220531T0317...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2022-05-31T03:17:15.582Z, Instrument: SA...",false,2022-05-31 03:17:15.582,2022-05-31 03:17:43.488,2022-05-31 06:07:00.067,340005,...,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,"<gml:Polygon srsName=""http://www.opengis.net/g...","MULTIPOLYGON (((38.860641 9.631741, 39.207008 ...",39f90581-99c2-4e50-843c-1d2c9ed2e5f8
f5634601-1c3b-49fa-895a-e4080143e4ca,S1A_IW_GRDH_1SDV_20220531T031716_20220531T0317...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2022-05-31T03:17:16.956Z, Instrument: SA...",false,2022-05-31 03:17:16.956,2022-05-31 03:17:41.955,2022-05-31 05:58:46.783,340005,...,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,"<gml:Polygon srsName=""http://www.opengis.net/g...","MULTIPOLYGON (((38.880753 9.724243, 39.193527 ...",f5634601-1c3b-49fa-895a-e4080143e4ca
735381e7-2f18-4f2f-bddc-def3b59c2b63,S1A_IW_SLC__1SDV_20220519T031714_20220519T0317...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2022-05-19T03:17:14.362Z, Instrument: SA...",false,2022-05-19 03:17:14.362,2022-05-19 03:17:42.268,2022-05-19 07:06:56.453,338676,...,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,"<gml:Polygon srsName=""http://www.opengis.net/g...","MULTIPOLYGON (((38.862846 9.632495, 39.209217 ...",735381e7-2f18-4f2f-bddc-def3b59c2b63
238124c8-74bd-4493-9181-eec9ec8cb97c,S1A_IW_GRDH_1SDV_20220519T031715_20220519T0317...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2022-05-19T03:17:15.749Z, Instrument: SA...",false,2022-05-19 03:17:15.749,2022-05-19 03:17:40.748,2022-05-19 06:58:17.537,338676,...,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,"<gml:Polygon srsName=""http://www.opengis.net/g...","MULTIPOLYGON (((38.882805 9.724217, 39.195652 ...",238124c8-74bd-4493-9181-eec9ec8cb97c
f3747324-f48f-4b66-ac84-914d6664fc92,S1A_IW_SLC__1SDV_20220507T031713_20220507T0317...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2022-05-07T03:17:13.583Z, Instrument: SA...",false,2022-05-07 03:17:13.583,2022-05-07 03:17:41.491,2022-05-07 06:29:41.760,337296,...,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,"<gml:Polygon srsName=""http://www.opengis.net/g...","MULTIPOLYGON (((38.862858 9.632538, 39.20927 1...",f3747324-f48f-4b66-ac84-914d6664fc92
2a88cf51-065c-4d58-8f1b-6fe162a0a864,S1A_IW_GRDH_1SDV_20220507T031714_20220507T0317...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2022-05-07T03:17:14.972Z, Instrument: SA...",false,2022-05-07 03:17:14.972,2022-05-07 03:17:39.969,2022-05-07 06:13:57.592,337296,...,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,"<gml:Polygon srsName=""http://www.opengis.net/g...","MULTIPOLYGON (((38.882828 9.724346, 39.195683 ...",2a88cf51-065c-4d58-8f1b-6fe162a0a864
c5695d8a-b167-4b80-a88a-8639770ba77d,S1A_IW_SLC__1SDV_20220425T031713_20220425T0317...,https://scihub.copernicus.eu/dhus/odata/v1/Pro..

In [127]:
s1_products_df[['orbitdirection','polarisationmode','sensoroperationalmode', 'relativeorbitnumber', 'orbitnumber', 'slicenumber']]

,orbitdirection,polarisationmode,sensoroperationalmode,relativeorbitnumber,orbitnumber,slicenumber
77a0735e-fcc7-472a-9fc8-c0832286693e,DESCENDING,VV VH,IW,152,45549,7
84008f80-4f5f-40b9-86d6-f73f3fb9eba8,DESCENDING,VV VH,IW,152,45549,7
0b22a3bf-9526-4bbd-8d92-4a7fb831fc11,DESCENDING,VV VH,IW,152,45374,7
024d7bef-0d9b-45a5-8f83-03c689ba5614,DESCENDING,VV VH,IW,152,45374,7
ae6b6c3d-15da-4cb6-be50-bb509aa2b5b9,DESCENDING,VV VH,IW,152,45199,7
effc1c2e-3e82-45ac-b263-ade2ab1a408e,DESCENDING,VV VH,IW,152,45199,7
089742aa-d3cc-4c65-9cf8-68ac2b4f19d2,DESCENDING,VV VH,IW,152,45024,7
eaf3be1e-9320-4db9-a5cd-99d985ddb919,DESCENDING,VV VH,IW,152,45024,7
16c7d6fb-cc72-4f57-8120-cd61096b9f50,DESCENDING,VV VH,IW,152,44849,7
a4175cd9-469f-447d-a182-0134ded2d6c2,DESCENDING,VV VH,IW,152,44849,7


In [110]:
np.unique(s1_products_df.slicenumber)

array([7], dtype=int64)

In [111]:
s1_products_df.polarisationmode

77a0735e-fcc7-472a-9fc8-c0832286693e    VV VH
84008f80-4f5f-40b9-86d6-f73f3fb9eba8    VV VH
0b22a3bf-9526-4bbd-8d92-4a7fb831fc11    VV VH
024d7bef-0d9b-45a5-8f83-03c689ba5614    VV VH
ae6b6c3d-15da-4cb6-be50-bb509aa2b5b9    VV VH
effc1c2e-3e82-45ac-b263-ade2ab1a408e    VV VH
089742aa-d3cc-4c65-9cf8-68ac2b4f19d2    VV VH
eaf3be1e-9320-4db9-a5cd-99d985ddb919    VV VH
16c7d6fb-cc72-4f57-8120-cd61096b9f50    VV VH
a4175cd9-469f-447d-a182-0134ded2d6c2    VV VH
9ac71d09-6cd3-4327-ab20-04b2b0fa3a3b    VV VH
b600c739-072e-4866-bd09-236e018e118b    VV VH
88b86a8c-c7c0-4822-b479-e4e0b4db9f59    VV VH
39c1f483-02d0-4371-9f8c-4222104b8fb2    VV VH
124ffbce-34dd-4062-9391-fbbd4a755205    VV VH
59ae5d35-c2a7-4e5a-bdfd-8d79713a39e2    VV VH
201ad05a-7774-4916-a258-7face0d3fb64    VV VH
ee191f6d-b635-47b1-b775-5d5635d71b99    VV VH
2e8cec2f-b8c7-4fcb-a7f1-bb8b60e48477    VV VH
e86aecd6-5150-49cd-b54c-95fa99cfcc7d    VV VH
be2956d2-d0b6-4946-a4c1-22f9d81af0d2    VV VH
ff30e56c-295e-49b7-8d07-925d762681

In [112]:
s1_products_df.keys()

Index(['title', 'link', 'link_alternative', 'link_icon', 'summary', 'ondemand',
       'beginposition', 'endposition', 'ingestiondate', 'missiondatatakeid',
       'slicenumber', 'orbitnumber', 'lastorbitnumber', 'relativeorbitnumber',
       'lastrelativeorbitnumber', 'filename', 'format', 'identifier', 'size',
       'status', 'timeliness', 'sensoroperationalmode', 'swathidentifier',
       'orbitdirection', 'producttype', 'platformname', 'platformidentifier',
       'instrumentname', 'instrumentshortname', 'productclass',
       'polarisationmode', 'acquisitiontype', 'gmlfootprint', 'footprint',
       'uuid'],
      dtype='object')

In [ ]:
############ Array Processing ####

In [80]:
##############################
# Planet workflwos
"""
1. Download
2. Mosaic
3. Reproject
4. Resample
5. Crop
"""

In [ ]:
def shp2gjson(infile, outfile):
    myshpfile = gpd.read_file(infile).to_crs("EPSG:4326")
    myshpfile.to_file(outfile, driver='GeoJSON')

infile = "/AOI/01_AOI_Salzachauen_A_Buffer.shp"
outfile = /AOI/01_AOI_Salzachauen_A_Buffer.geojson"
shp2gjson(infile=infile, outfile=outfile)

In [3]:
class Planethandler:
    def __init__(self,
                 year=2022,
                 months = [1, 12],
                 aoi = '/footprint/explorer-aoi_final.geojson',
                 key='f98908e6f47f44d3b682564677db5d95',
                 product = 'planet_medres_normalized_analytic_',
                 path='/Planet_Scope_Analytic_second'):
        self.year = year
        self.months = [str(i) for i in range(months[0], months[1]+1)]
        self.aoi = aoi
        self.key = key
        self.url = "https://api.planet.com/basemaps/v1/mosaics"
        self.product = product  # either of planet_medres_normalized_analytic_2022-06_mosaic or planet_medres_visual_2022-06_mosaic
        self.session = requests.Session()
        self.session.auth = (self.key, "")
        self.path = path
        
    def download(self):
        for month in self.months:
            if int(month)<10:
                tile = f"{self.product}{str(self.year)}-0{month}_mosaic"
            else:
                tile = f"{self.product}{str(self.year)}-{month}_mosaic"
            
            parameters = {"name__is" :tile}
            res = self.session.get(self.url, params = parameters)
            if res.status_code != 200:
                print('Session did not connected with server, please check API URL and authentication')
                
            mosaic = res.json()
#             print(mosaic)
            mosaic_id = mosaic['mosaics'][0]['id']
#             print(mosaic['mosaics'][0].keys())
            
            df = gpd.read_file(self.aoi)
            bbox = [df.bounds['minx'][0], df.bounds['miny'][0], df.bounds['maxx'][0], df.bounds['maxy'][0]]
            new_bbox = ','.join(map(str, bbox))
            
            search_parameters = {
                'bbox': new_bbox,
                'minimal': True}
            quads_url = "{}/{}/quads".format(self.url, mosaic_id)
            res = self.session.get(quads_url, params=search_parameters, stream=True)
            
            quads = res.json()
#             print(quads)
            items = quads['items']
            print(f'Downloading {tile}', end='\r', flush=True)
            
            DIR = f'{self.path}/{tile}'
            if not os.path.exists(DIR):
                os.makedirs(DIR, exist_ok=True)
            for i in items:
                link = i['_links']['download']
                name = i['id']
                name = name + '.tiff'
                filename = f'{DIR}/{name}'
                print(link)
                urllib.request.urlretrieve(link, filename)

In [ ]:
# defaults
Planethandler()

In [ ]:
# user specific inputs
downloader = Planethandler(year=2022,
                 months = [1, 12],
                 aoi = aoi_salzburg,
                 key='f98908e6f47f44d3b682564677db5d95',
                 product = 'planet_medres_normalized_analytic_',
                 path='path to save')
downloader.download()

In [6]:
# combined predicted individual tiles to larger scene
def moasic(in_root=None, out_root=None, name=None, remove_tiles=False, pattern=None, rootfiles = None, resample=True, scale=2):
    print('preparing mosaic..',  end='\r', flush=True)
    if in_root is not None:
        if pattern is None:
            files = [in_root + f'/{file}' for file in os.listdir(in_root)]
        else:
            files = [in_root + f'/{file}' for file in os.listdir(in_root) if pattern in file and file[-4:] == '.tif']
    else:
        files = rootfiles
    
    opened = [] # container for all opened files in read mode
    for file in files:
        opened.append(rasterio.open(file))
    
    print('creating mosaic..',  end='\r', flush=True)
    mosaic, out_trans = merge(opened, method='max') # method='max' # this is mainly to overcome shadows and cloud contaminated pixels
    out_meta = opened[0].profile
    if resample:
        new_affine =  affine.Affine(out_trans.a*1/scale,
                                    out_trans.b,
                                    out_trans.c,
                                    out_trans.d,
                                    out_trans.e*1/scale,
                                    out_trans.f)
        
        out_meta.update({"driver": "GTiff",
                         "dtype":np.float64,
                         "height": mosaic.shape[1]*scale,
                         "width": mosaic.shape[2]*scale,
                         "transform": new_affine,
                         "crs": out_meta['crs']
                        })
        
    else:
        out_meta.update({"dtype":mosaic.dtype,
                         "height": mosaic.shape[1],
                         "width": mosaic.shape[2],
                         "transform": out_trans,
                        })
    if name is None:
        name = os.path.split(files[0])[1]
    masoic_name = out_root + f'/{name}.tif'
    
    print('writting a mosaic',  end='\r', flush=True)
    with rasterio.open(masoic_name, "w", **out_meta) as dest:
        if resample:
            for i in range(1, 5):
                dst = rescale(mosaic[i-1],
                             2,
                             clip=False,
                             preserve_range=True,
                             anti_aliasing=False,
                             anti_aliasing_sigma=False)
                dest.write(dst, i)  # .astype(out_meta.dtype)
        else:
            for i in range(1, mosaic.shape[0]+1):
                dest.write(mosaic[i-1], i)
        
    if remove_tiles:
        print('Removing tiles',  end='\r', flush=True)
        del opened
        files_2_del = [rasterio.open(root + '/' + file) for file in os.listdir(root) if 'mosaic' not in file]
        for files in files_2_del:
            os.remove(file)

In [7]:
class Mosaic:
    def __init__(self, dataset='planet'):
        self.dataset = dataset
    def do_mosaic(self, root_in=None, root_out=None):
        if self.dataset == 'planet' and root_in is not None:
            folds = os.listdir(root_in)
            for tile_fold in folds:
                image_dir = root_in + f'/{tile_fold}'
                file_name = tile_fold
                if root_out is None:
                    root_out = os.path.split(root_in)[0] + '/Mosaics'
                if not os.path.exists(root_out):
                    os.makedirs(root_out, exist_ok=True)
                    
                moasic(in_root=image_dir,
                       out_root=root_out,
                       name=file_name,
                       remove_tiles=False)
                print('Done! Mosaics are saved in {}'.format(root_out))
                
        elif self.dataset == 'skysat':
            folds = os.listdir(root_in)
            for tile_fold in folds:
                image_dir = root_in + f'/{tile_fold}/files'
                file_name = tile_fold
                if root_out is None:
                    root_out = os.path.split(root_in)[0] + '/Mosaics'
                if not os.path.exists(root_out):
                    os.makedirs(root_out, exist_ok=True)
                moasic(in_root=image_dir,
                       out_root=root_out,
                       name=file_name,
                       remove_tiles=False,
                       pattern='analytic_SR_clip')
                print('Done! Mosaics are saved in {}'.format(root_out))
        elif self.dataset == 'sentinel-2':
            if root_out is None:
                root_out = os.path.split(root_in)[0] + '/Mosaics'
                
            if not os.path.exists(root_out):
                os.makedirs(root_out, exist_ok=True)
                    
            image_pairs =  return_matchs(root_in)
            print(image_pairs)
            for j, pair in enumerate(image_pairs):
                pair = sorted(pair)
                name = os.path.splitext(os.path.split(pair[0])[1])[0]
                print(name)
                moasic(out_root=root_out, name=name, rootfiles=pair)  # needs readjustment
        else:
            pass

In [23]:
data_path = '/Planet_Scope_Analytic_second/Temporal_tiles'
sticher = Mosaic(dataset='planet')
out_fold = '/Planet_Scope_Analytic_second/Crops'
sticher.do_mosaic(root_in=data_path, root_out=out_fold)

Done! Mosaics are saved in F:/EO_Africa_DAT/Planet_Scope_Analytic_second/Crops
Done! Mosaics are saved in F:/EO_Africa_DAT/Planet_Scope_Analytic_second/Crops
Done! Mosaics are saved in F:/EO_Africa_DAT/Planet_Scope_Analytic_second/Crops


In [ ]:
# CRS.from_epsg(32637)
# skysat: CRS.from_epsg(32637)
# sentinel-2: CRS.from_epsg(32637)

In [68]:
# Sentinel data preprocessing python
def unzip_sentinel2(input_dir):
    save_dir = input_dir + f'/Temporal_tiles'
    if not os.path.exists(input_dir):
        os.makedirs(save_dir, exist_ok=True)
        
    files = [input_dir + f'/{ziped}' for ziped in os.listdir(input_dir)]
    out_folders = [os.path.split(input_dir)[0] + f'/Temporal_tiles/{os.path.split(path)[1][:-4]}' for path in files]
    
    pairs = list(zip(files,out_folders))
    for j, data in enumerate(pairs):
        print(f'Unziping >> {j} th product...', end='\r', flush=True)
        with zipfile.ZipFile(data[0], 'r') as zip_ref:
            zip_ref.extractall(data[1])

In [69]:
def unzip_TerraSAR(input_dir):
    save_dir = input_dir + f'/Temporal_tiles'
    if not os.path.exists(input_dir):
        os.makedirs(save_dir, exist_ok=True)
    files = [input_dir + f'/{ziped}' for ziped in os.listdir(input_dir) if 'tar.gz' in ziped]
    out_folders = [os.path.split(input_dir)[0] + f'/Temporal_tiles/{os.path.split(path)[1][:-7]}' for path in files]
    
    pairs = list(zip(files,out_folders))
    for j, data in enumerate(pairs):
        print(f'Unziping >> {j} th product...', end='\r', flush=True)
        with tarfile.open(data[0], 'r') as tar_open:
            tar_open.extractall(data[1])

In [56]:
dirr = '/Sentinel-2_seond/Data'
unzip_sentinel2(dirr)

In [4]:
dirr = '/TeraSAR-X/aa'
unzip_TerraSAR(dirr)

In [70]:
def reprojectRaster(input_rst, out_folder=None, from_epsg=32637):
    dst_crs = f'EPSG:{from_epsg}'
    with rasterio.open(input_rst) as src:
        transform, width, height = calculate_default_transform(src.crs,
                                                               dst_crs,
                                                               src.width,
                                                               src.height,
                                                               *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({'crs': dst_crs,
                       'transform': transform,
                       'width': width,
                       'height': height
                      })
        if out_folder is None:
            out_folder = os.path.split(os.path.split(input_rst)[0])[0] + '/Reprojeted_files'
        if not os.path.exists(out_folder):
            os.makedirs(out_folder, exist_ok=True)
            
        out_file_path = out_folder + f'/{os.path.split(input_rst)[1]}'
        
        with rasterio.open(out_file_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(source=rasterio.band(src, i),
                          destination=rasterio.band(dst, i),
                          src_transform=src.transform,
                          src_crs=src.crs,
                          dst_transform=transform,
                          dst_crs=dst_crs,
                          resampling=Resampling.nearest)

In [25]:
path = '/Planet_Scope_Analytic_second/Mosaics'
files = glob(f'{path}/*.tif')
for file in files:
    reprojectRaster(input_rst=file, out_folder=None, from_epsg=32637)

In [71]:
def resamplRaster(input_rst, scale_factor=2, resampling=Resampling.bilinear, out_folder=None, report=False):
    if not os.path.exists(out_folder):
        os.makedirs(out_folder, exist_ok=True)
    with rasterio.open(input_rst) as dataset:
        data = dataset.read(
            out_shape=(
                dataset.count,
                round(dataset.height * scale_factor),
                round(dataset.width * scale_factor)
            ),
            resampling=resampling
        )
        
        if report:
            print('Shape before resample:', dataset.shape)
            print('Shape after resample:', data.shape[1:])

        # scale image transform
        dst_transform = dataset.transform * dataset.transform.scale(
            (dataset.width / data.shape[-1]),
            (dataset.height / data.shape[-2])
        )
        if report:
            print('Transform before resample:\n', dataset.transform, '\n')
            print('Transform after resample:\n', dst_transform)

    ## Write outputs
    ## set properties for output
        dst_kwargs = dataset.meta.copy()
        dst_kwargs.update(
            {"transform": dst_transform,
             "width": data.shape[-1],
             "height": data.shape[-2],
             "nodata": 0,
            }
        )
        save_path = out_folder + '/' + os.path.split(input_rst)[1]
        with rasterio.open(save_path, "w", **dst_kwargs) as dst:
            for i in range(data.shape[0]):
                  dst.write(data[i].astype(data.dtype), i+1)

In [30]:
scale = 4.67283176945198/5
out_fold = '/Planet_Scope_Analytic_second/Resampled'
path = '/Planet_Scope_Analytic_second/Reprojeted_files'
files = glob(f'{path}/*.tif')
for file in files:
    resamplRaster(input_rst=file, scale_factor=scale, resampling=Resampling.bilinear, out_folder=out_fold, report=True)

Shape before resample: (8234, 12364)
Shape after resample: (7695, 11555)
Transform before resample:
 | 4.67, 0.00, 272351.15|
| 0.00,-4.67, 1293697.11|
| 0.00, 0.00, 1.00| 

Transform after resample:
 | 5.00, 0.00, 272351.15|
| 0.00,-5.00, 1293697.11|
| 0.00, 0.00, 1.00|
Shape before resample: (8234, 12364)
Shape after resample: (7695, 11555)
Transform before resample:
 | 4.67, 0.00, 272351.15|
| 0.00,-4.67, 1293697.11|
| 0.00, 0.00, 1.00| 

Transform after resample:
 | 5.00, 0.00, 272351.15|
| 0.00,-5.00, 1293697.11|
| 0.00, 0.00, 1.00|
Shape before resample: (8234, 12364)
Shape after resample: (7695, 11555)
Transform before resample:
 | 4.67, 0.00, 272351.15|
| 0.00,-4.67, 1293697.11|
| 0.00, 0.00, 1.00| 

Transform after resample:
 | 5.00, 0.00, 272351.15|
| 0.00,-5.00, 1293697.11|
| 0.00, 0.00, 1.00|


In [72]:
def sentinel2CloudMask(inpath='F:/EO_Africa_DAT/Sentinel-2/Temporal_tiles', outpath='F:/EO_Africa_DAT/Sentinel-2/Cloud_MaskedRGB'):
    if not os.path.exists(outpath):
        os.makedirs(outpath, exist_ok=True)
    for j, im_folder in enumerate(os.listdir(inpath)):
#         if j == 0:
        print(f'Processing {im_folder} ...', end='\r', flush=True)
        xml_file = inpath + f'/{im_folder}/{im_folder}.SAFE/MTD_MSIL2A.xml'
#         print(xml_file)

        tree = ET.parse(xml_file)
        root = tree.getroot()

        ext = '.jp2'

#         if  root[0][0][11][0][0] == 'Product_Organisation':
#             handle = root[0][0][11][0][0]
#         elif root[0][0][12][0][0] == 'Product_Organisation':
#             handle = root[0][0][11][0][0]
#         else:
#             raise ValueError('Handle note found')

        try:
            handle = root[0][0][11][0][0]
        except:
            print('Looking alternative paths')
        finally:
            if 'handle' in locals():
                pass
            else:
                handle = root[0][0][12][0][0]

        im_format = handle.attrib['imageFormat']
        texts = []
        for i in range(len(handle)):
            a = handle[i].text
            if ('R10m' in a and '_B0' in a) or ('_SCL_20m' in a):
                texts.append(a+ext)
                
        assert texts is not None, 'The images are not properly read'
        del handle   

        scl = os.path.split(xml_file)[0] + f'/{texts[-1]}' # scene classification layer
        rst = rasterio.open(scl).read()       # scene classification layer
        mask = np.empty(rst.shape, np.bool)  # empty mask

        del tree
        del root 

        inds = list(range(1, 12))
        for ind in inds:
            if ind in [3, 8, 9, 10]:
                mask[rst==ind] = False
            else:
                mask[rst==ind] = True
        mask = np.expand_dims(rescale(mask[0],
                       2,
                       clip=False,
                       preserve_range=True,
                       anti_aliasing=False,
                       anti_aliasing_sigma=False), axis=0)
        print(f'shape of resampled shape: {mask.shape}')
#         print(texts)
        
        flighter = rasterio.open(scl)
        left, bottom, right, top = flighter.bounds
        new_affine =  affine.Affine(10, 0.0, left, 0.0, -10, top)
        profile = flighter.profile
        profile['dtype'] = np.uint16
        profile['width'] = profile['width']*2
        profile['height'] = profile['height']*2
        profile['blockxsize'] = 1024
        profile['blockysize'] = 1024
        profile['count'] = 4
        profile['driver'] = 'GTiff'
        profile['transform'] = new_affine
        flighter.close()
        del flighter

        file_to_save = outpath + '/' + im_folder + '.tif'

        with rasterio.open(file_to_save, 'w', **profile) as dst:
            for i,sample in enumerate(texts[:-1]):
                sample_path = os.path.split(xml_file)[0] + f'/{sample}'
                im = rasterio.open(sample_path).read()
                new_dtype = im.dtype
                im = (im*mask.astype(new_dtype)).squeeze()
                dst.write(im.astype(new_dtype), i+1) 

In [73]:
# Mosaicing sentinel-2
def fetch_bbox(raster, save=False, plot=False, out_dir=None, return_val=True):
    if out_dir is not None:
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)
    else:
        pass
    with rasterio.open(raster) as src:
        geom = box(*src.bounds)
        gdf = gpd.GeoDataFrame({"id":1,"geometry":[geom]}).set_crs(src.crs)
        if save:
            if out_dir is None:
                out_dir = os.getcwd()
            name = os.path.splitext(os.path.split(raster)[1])[0]
            name = out_dir + f'/{name}.shp'
            gdf.to_file(name)
        if plot:
            gdf.plot()
        if return_val:
            return gdf
        else:
            pass

In [74]:
def return_matchs(data_dir, ext='.tif'):
    rasters =  glob(f'{data_dir}/*{ext}')
    PCN = [file for file in rasters if 'PCN' in file]
    PBN = [file for file in rasters if 'PBN' in file]
    
    pcn = [] # with match
    pbn = [] # with match
    
    for i, file in enumerate(PCN):
        ma = os.path.split(file)[1][11:19]
        for j, match in enumerate(PBN):
            if ma in match:
                pcn.append(file)
                pbn.append(match)
    pcn_l = list(set(PCN).symmetric_difference(set(pcn)))
    pbn_l = list(set(PBN).symmetric_difference(set(pbn)))
    
    pcn_l = list(set(PCN).symmetric_difference(set(pcn)))
    pbn_l = list(set(PBN).symmetric_difference(set(pbn)))
    
    match2 = []  # 

    for file in pbn_l:
        tlags = []
        date1 = os.path.split(file)[1][11:19]
        time1 = datetime.datetime.strptime(date1, '%Y%m%d') # extract date from both

        for file2 in pcn_l:
            date2 =  os.path.split(file2)[1][11:19]
            time2 = datetime.datetime.strptime(date2, '%Y%m%d')
            time_dif = abs((time2-time1).days)  # compute date diference
            tlags.append(time_dif)
        ind = tlags.index(min(tlags))
        match2.append((file, pcn_l[ind]))
        
    match1 = list(zip(pbn, pcn))

    all_match = match1 + match2
    
    return all_match

In [75]:
def cropRaster(raster,
               out_fold,
               shape='/Sky_SAT/footprint/explorer-aoi_final_projected.geojson',
               resample=False,
               scale=2, index=None):
    with fiona.open(shape, "r") as shapefile:
        geoms = [feature["geometry"] for feature in shapefile]
#     geoms = gdf['geometry']
    if not os.path.exists(out_fold):
        os.makedirs(out_fold, exist_ok=True)
    with rasterio.open(raster) as src:
        if index is None:
            out_image, out_transform = rasterio.mask.mask(src, geoms, crop=True)
        else:
            out_image, out_transform = rasterio.mask.mask(src, geoms, crop=True, indexes=index)
            out_image[out_image==0] = np.nan
        out_meta = src.meta
        if resample:
            new_affine =  affine.Affine(out_transform.a*1/scale,
                                    out_transform.b,
                                    out_transform.c,
                                    out_transform.d,
                                    out_transform.e*1/scale,
                                    out_transform.f)
            
            out_meta.update({"driver": "GTiff",
                             "height": out_image.shape[1]*scale,
                             "width": out_image.shape[2]*scale,
                             "transform": new_affine,
                             "crs": out_meta['crs']
                            })
        else:
            out_meta.update({"driver": "GTiff",
                             "height": out_image.shape[1],
                             "width": out_image.shape[2],
                             "nodata":out_image.shape[0],
                             "count":4,
                             "dtype":np.uint16,
                             "transform": out_transform})
        out_name = out_fold + '/' + os.path.split(raster)[1]
        
        with rasterio.open(out_name, "w", **out_meta) as dest:
            if resample:
                for i in range(0, 4):
                    dst = rescale(out_image[i],
                                  2,
                                  clip=False,
                                  preserve_range=True,
                                  anti_aliasing=False,
                                  anti_aliasing_sigma=False)
                    dest.write(dst, i+1)
                print(f'Done for {out_name}', end='\r', flush=True)
            else:
                dest.write(out_image.astype(np.uint16))

In [76]:
def batch_crop(in_fold, out_fold):
    if not os.path.exists(out_fold):
        os.makedirs(out_fold, exist_ok=True)
    imgs = [in_fold + f'/{img}' for img in os.listdir(in_fold)]
    #print(imgs)
    #cropRaster(raster=image, out_fold=out_fold)
    for image in imgs:
        try:
            cropRaster(raster=image,
                       out_fold=out_fold)
        except:
            pass

In [1]:
ins = '/Planet_Scope_Analytic_second/Resampled'
outs = '/Planet_Scope_Analytic_second/Crops'
batch_crop(in_fold=ins, out_fold=outs)

In [77]:
def coregisterRaster(slave, master, out_folder, report=False):
    with rasterio.open(slave) as slave_src:
        src_transform = slave_src.transform
        
        with rasterio.open(master) as master_src:
            dst_crs = master_src.crs
        
            dst_transform, dst_width, dst_height = calculate_default_transform(
                slave_src.crs,    
                dst_crs,    
                master_src.width,   
                master_src.height,   
                *master_src.bounds)

        dst_kwargs = slave_src.meta.copy()
        dst_kwargs.update({"crs": dst_crs,
                           "transform": dst_transform,
                           "width": dst_width,
                           "height": dst_height,
                           "nodata": 0})  # needs some thought
        
        if report:
            print("New shape of an image:", dst_height,dst_width,'\n Affine parameters',dst_transform)

        outfile = out_folder + '/coreg_' + os.path.split(slave)[1]
        with rasterio.open(outfile, "w", **dst_kwargs) as dst:
            for i in range(1, slave_src.count + 1):
                reproject(
                    source=rasterio.band(slave_src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=slave_src.transform,
                    src_crs=slave_src.crs,
                    dst_transform=dst_transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)

In [ ]:
# UNZIP Sentinel-2
## SENTINEL WORKFLOW ################
"""
1. Downlaod
2. Unzip
3. Cloud mask
4. Crop
5. Moasic + resample or 
6. Resample
7. Coregister [optional]
"""
# root = '/Sentinel-2_second/Data'
# downloadSentinel(aoi=None, path2save=root)
# unzip_sentinel2(root)
# sentinel2CloudMask(inpath, outpath)
# cropRaster(raster=pair, out_fold=outs)
# maosicer = Mosaic(dataset='sentinel-2')
# maosicer.do_mosaic(root_in=ins, root_out=outs)